#pipelines

Pipelines change together multiple steps so that output of each step is used as input to the next step


Pipeline spake is easy to apply the same processing to train and test!


In [3]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [4]:
df=pd.read_csv('/content/train.csv')

#Without using pipelines

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [7]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
138,0,3,male,16.0,0,0,9.2167,S
849,1,1,female,NaN,1,0,89.1042,C
431,1,3,female,NaN,1,0,16.1000,S
349,0,3,male,42.0,0,0,8.6625,S
616,0,3,male,34.0,1,1,14.4000,S


In [8]:
#survived is the targer column

In [9]:
#step one train test split
X_train,X_test,Y_train,Y_test=train_test_split(df.drop('Survived',axis=1),df['Survived'],test_size=0.2,random_state=42)

In [10]:
Y_train.head()

,Survived
331,0
733,0
382,0
704,0
813,0


In [11]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [12]:
#apply imputations
si_age=SimpleImputer()
si_embarked=SimpleImputer(strategy='most_frequent')

X_train_age=si_age.fit_transform(X_train[['Age']])
X_train_embarked=si_embarked.fit_transform(X_train[['Embarked']])

X_test_age=si_age.fit_transform(X_test[['Age']])
X_test_embarked=si_embarked.fit_transform(X_test[['Embarked']])



In [13]:
#one hot ecoding
ohe_sex=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_sex=ohe_sex.fit_transform(X_train[['Sex']])
# Use the same ohe_embarked fitted on the training data to transform the test data
X_train_embarked=ohe_embarked.fit_transform(X_train[['Embarked']])

X_test_sex=ohe_sex.transform(X_test[['Sex']])
X_test_embarked=ohe_embarked.transform(X_test[['Embarked']])

Sparse is changes by sparse_output


In [14]:
X_train_rem=X_train.drop(['Sex','Age','Embarked'],axis=1)
X_test_rem=X_test.drop(['Sex','Age','Embarked'],axis=1)

In [15]:
#concatinationg all the numpy arrays

X_train_transformed=np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed=np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [16]:
X_train_transformed.shape

(712, 11)

multiconiliarity ko hatane ke liye column driop kyu nahi kiya ?-- because yaha decision tree use ho raha hai toh waha koi need nahi , liner model mai karna zaroori hunda si

In [17]:
clf=DecisionTreeClassifier()

In [18]:
clf.fit(X_train_transformed,Y_train)

DecisionTreeClassifier()

In [19]:
Y_pred=clf.predict(X_test_transformed)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.7486033519553073

In [21]:
import pickle
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf, open('decision_tree_model.pkl', 'wb'))


for train we have to load the above models and then do all the same test with the custon test input same test

thats why we use pipelines in this becuse production code mai bhi same steps aate hai , agar ham test code mi changes karege toh test code mai bhi changes karne padega -- not n efficoent way

#with pipelines

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [23]:

df=pd.read_csv('/content/train.csv')

In [24]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Lets plan

impute missing values--> ohe(sex , embarked)-->scaling(similar numerical range)-->Feature selection-->trining (decision tree)

In [25]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [26]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [27]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


#steps before implimentation of pipeline

In [28]:
#train test split
x_train,x_test,y_train,y_test=train_test_split(df.drop(['Survived'],axis=1),df['Survived'],test_size=0.2,random_state=42)

In [30]:
#Ist part of pipeline ie imputstion
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])

],remainder='passthrough')

 #yaha pe name ke wajay index numer diya hai beacuse yeh numpy hoga aur code fatne lagege and pipelines mai preffer karo ki index or numerical values he rahe input mai )

In [32]:
#2nd part id one hot encoding

trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [34]:
#3rd part is scaling
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

#after ohe 10 columns banjayge and sabpe scaling hogi thats why slice(0,10)
#slice used beacause wo iss range mai sab pe labeling karedega
#and minmax scaler isme beacuse feature selection karni hai warne we can se standrd scaling

In [35]:
#4th part is feature selection

trf4=SelectKBest(score_func=chi2,k=8)
#k here signifief the top 8 features gonna be selected


In [38]:
#5th part is train the model

trf5=DecisionTreeClassifier()

#creating pipeline

In [39]:
#implementiong pipeline

pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
     ('trf3',trf3),
    ('trf4',trf4),
     ('trf5',trf5)

])

#pipeline vs make_pipeline


pipelines require naming of steps , make_pipeline does not
make_pipeline mai hame toh names nahi pass karne rehte hau sidha tupple object pass kardo


(same applies for columntransformer and make_column_transformer

In [40]:
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [45]:
pipe.fit(X_train,Y_train)

#imp-->ohe-->scaling--> if we havent train a model then we will use fit_transform(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7d571c83d1c0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [44]:
#display pipeline
from sklearn import set_config
set_config(display='diagram')

#exploring the pipelines

In [57]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

#stststics for mean value
   # we can check the total steps involved and then specific name deke bhi check kar sakte uss transformer mai kya kaam hua hai

array([29.49884615])

In [59]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

# this command will use to check the parameter and helps to debugging

array(['S'], dtype=object)

In [61]:
Y_predict=pipe.predict(X_test)

In [63]:
Y_predict

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_predict)

0.6256983240223464

#crossvalidations using pipelines

In [68]:
#watch after this concept

#export

In [82]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [83]:
test_input2=np.array([2,'male',31.0,0,0,10.5,"S"])

In [92]:
test_input2 = np.array([[2,'female',31.0,0,0,10.5,'C']])
pipe.predict(test_input2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])